In [1]:
from lxml import etree
from collections import namedtuple
from tqdm import tqdm, tnrange, tqdm_notebook
import json
import pandas as pd
import utils
import os


In [2]:
# sentences = []
# c=0
# for sentence in tqdm(Training.parse()):
#     c+=1
#     sentences.append([i.lemma for i in sentence])
#     break
!ls ../resources/WSD_Evaluation_Framework/Training_Corpora/SemCor

semcor.data.xml     semcor.gold.key.txt


# Gold

In [ ]:
class GoldParser(object):
    """
    Class to parse the txt gold files
    """
    def __init__(self, file):
        self.file = file
        self.records = namedtuple("GoldEntry", "id_ senses")
    def parse(self):
        
        with open(self.file, mode="r") as goldfile:
            for line in goldfile:
                line = line.strip().split(" ")
                entry = self.records(id_ = line[0],
                                     senses = list({utils.sensekeyToSynsetConverter(i)
                                                    for i in line[1:]}))
                yield entry

In [ ]:
#Gold = GoldParser('../resources/training-data/WSD_Training_Corpora/SemCor+OMSTI/semcor+omsti.gold.key.txt')
#Gold = GoldParser('../resources/training-data/WSD_Training_Corpora/SemCor/semcor.gold.key.txt')

In [ ]:
# c = 0
# entries = []
# for entry in tqdm(Gold.parse()):
#     c+=1
#     entries.append(entry)


In [ ]:
#entries = [i for i in tqdm(Gold.parse())]
    

In [ ]:
class EvaluationParser(object):
    """
    Class to parse the txt evaluation files
    """
    def __init__(self, gold_path):
        self.gold_path = gold_path
        
    @staticmethod
    def read_mapping(file):
        """
        reads current mapping and returns a dict
        """
        with open(file, mode="r") as f:
            mapping = {}
            for line in f:
                line = line.strip().split("\t")
                key, value = line[:2]
                mapping.update({value:key})
        return mapping
    
    def to_babelnet(self, input_path, output_path):
        """
        WordNet to BabelNet
        """
        current_mapping = self.read_mapping(input_path)
        
        with open(self.gold_path, 'r') as f, open(output_path, 'w') as out:
            
            for line in tqdm(f, "WordNet to BabelNet"):
                line = line.strip().split(" ")
                if len(line)<2:
                    continue
                id_, sense_key = line[:2]  
                wn_id = utils.sensekeyToSynsetConverter(sense_key)
                fmt = "{} {}\n".format(id_, current_mapping[wn_id])
                out.write(fmt)
                out.flush()
                
    def to_coarse_grained(self, input_path, output_path, babelnet_path, type_='LEX'):
        """
        BabelNet to WordNet Domain or LEX
        :param babelnet_path: babelnet mapping
        :param type_: LEX or WND
        """
        current_mapping = self.read_mapping(input_path)
        current_mapping = dict(zip(current_mapping.values(), current_mapping.keys()))
        with open(babelnet_path, 'r') as f, open(output_path, 'w') as out:
            for line in tqdm(f, "BabelNet to {}".format(type_)):
                line = line.strip().split(" ")
                if len(line)<2:
                    continue
                id_, bn_id = line[:2] 
                if bn_id in current_mapping.keys():
                    new = current_mapping[bn_id]
                    print(new)
                else:
                    new = '<missing-' + type_ + '>'
                
                fmt = "{} {}\n".format(id_, new)
                out.write(fmt)
                out.flush()
    

In [ ]:

def listdir_fullpath(d):
    return [os.path.join(d, f) for f in os.listdir(d)]

dir_ = "../resources/WSD_Evaluation_Framework/Evaluation_Datasets"
eval_datasets = [i for i in os.listdir(dir_) if i.startswith("se")]
eval_datasets

In [ ]:
for name in eval_datasets:
    print("Dataset: {}".format(name))
    
    path = os.path.join(dir_, name)
    gold_file = [i for i in os.listdir(path) if i.endswith('gold.key.txt')][0]
    gold_file = os.path.join(path, gold_file)
    print("using {}".format(gold_file))
    Eval = EvaluationParser(gold_file)
    
    bn_output_path = os.path.join(path, 'gold_BN.txt')
    print(bn_output_path)
    Eval.to_babelnet(input_path = '../resources/babelnet2wordnet.tsv',
                     output_path = bn_output_path)
    
    
    #to WordNetDomain
    Eval.to_coarse_grained(input_path = '../resources/babelnet2wndomains.tsv',
                          output_path = os.path.join(path, 'gold_WND.txt'),
                          babelnet_path = bn_output_path,
                          type_='WND')
    
    #to LEX
    Eval.to_coarse_grained(input_path = '../resources/babelnet2lexnames.tsv',
                          output_path = os.path.join(path, 'gold_LEX.txt'),
                          babelnet_path = bn_output_path,
                          type_='LEX')
    
    
#     df = parse_me(gold_file = gold_file,
#              babelnet2wordnet = '../resources/babelnet2wordnet.tsv',
#              babelnet2wndomains = '../resources/babelnet2wndomains.tsv',
#              babelnet2lexnames = '../resources/babelnet2lexnames.tsv')
    
    
    